# ESD 40.317 Financial Systems Design 
## Part 2 Assignment 4
### Due: 16 August before midnight 

Do not change any of the function definitions or variable names that are given in the cells for you to provide the solution. These function and variable names are used to autograde your solution and changing them might cause the autograder to fail and hence award 0 marks.

We will be continuing with the Constant Elasticity of Variance (CEV) model for risk-neutral stock price evolution with $\beta = 0.75$. Recall the SDE:    

$$dX = rXdt + σX^β dW$$

We will be computing the price of the same European up-and-in call option but with Monte Carlo simulations. The terms are: expiration in 0.5 years, strike price of \\$100 and EKI barrier at \\$115 which is only observed at maturity. The current stock price, $x_0$, is \\$110 with a volatility of 25\%. The interest rate is constant at 2\%.


In [1]:
import math
import numpy as np
from scipy.stats import norm

X0 = 110.; K = 100; T = 0.5; r = 0.02; σ = 0.25; B = 115; β = 0.75  # parameters

### a) Milstein Scheme

Use the Milstein scheme to simulate the price path of the underlying asset. For the CEV process, the scheme will be

$X_{t+\Delta t}  = X_t + rX_t \Delta t + \sigma X_t^β \sqrt{\Delta t} Z+\frac{1}{2}\beta \sigma^2 X_t^{2β-1} \Delta t(Z^2-1)$ where $Z~N(0,1)$

Complete the **milstein** function definition below that generates N number of stock price paths using the Milstein scheme under the CEV model. The function takes in the following arguments:
1. X0 is the initial stock price
2. T is the duration for the price path
3. $\sigma$ is the volatility
4. r is the interest rate
5. $\beta$ is the elasticity of variance with respect to X
6. N is the total number of simulations
7. steps is the number of equally spaced intervals to reach time T

The function should return a numpy array of shape (N, ). Each element represents the final price of the simulated path for the stock price starting from X0.

(20 marks)

In [2]:
def milstein(X0, T, σ, r, β, N, steps):
    # ADD CODE HERE
    Δt = T/steps
    X = X0*np.ones(N)
    prng = np.random.RandomState() 
    z = prng.standard_normal((N, steps)) # create an array of randomly generated numbers. rows for # of paths, col for steps
    for t in range(steps):
        X += r*X*Δt + σ*X**β*math.sqrt(Δt)*z[:,t] + 0.5*β*σ**2*X**(2*β-1)*Δt*(np.power(z[:,t],2)-1)
    
    return X
# S0 = 100
# test1 = S0*np.ones(10)
# print(test1)
# prng = np.random.RandomState(5) 
# Z = prng.standard_normal((10,5))
# Y = np.linspace(1,10,10)
# print(Z)
# print(Z[:,1]*test1)
# print(X1.shape)

In [3]:
%%time

X1 = milstein(X0, T, σ, 0.2, β, 500000, 100)

assert X1.shape == (500000,)

Wall time: 21.9 s


### b) Standard Monte Carlo

Complete the **eki_call** function definition below that will be used to find the price of the option using the Monte Carlo method.

(20 marks)

In [4]:
def eki_call(X0, K, B, T, σ, r, β, N, steps):
    X = milstein(X0, T, σ, r, β, N, steps)
    # ADD CODE HERE
    # at this point, we have the asset price at maturity for each path
    KI = X>B
    V = math.exp(-r*T)*np.maximum(X-K,0)*KI
    
    # process the price
    price = np.average(V)
    
    # process the stderr
    stderr = np.std(V, ddof=1) / N**0.5
    
    return price, stderr

# print(math.fabs(price1 - 4.77) < 4e-2)

In [5]:
%%time

price1, stderr1 = eki_call(X0, K, B, T, σ, r, β, 500000, 100)
assert math.fabs(price1 - 4.77) < 4e-2

Wall time: 21.7 s


Complete the **eki_call_control** function definition below that will be used to find the price of the option using the Monte Carlo method with the additional use of a control variable. The control variable will be the price of a European vanilla call option under the CEV model. The function **cev_vanilla_call** to calculate the price of the vanilla call under the CEV model is provided for you in the cell below.

(20 marks)

In [6]:
from scipy.stats import ncx2

def cev_vanilla_call(X0, K, T, σ, r, β):
    b = β - 1
    n = 2 + 1 / math.fabs(b)
    p = 2*r*X0**(-2*b) / (σ**2*b*(math.exp(2*r*b*T) - 1))
    d = 2*r*K**(-2*b) / (σ**2*b*(1 - math.exp(-2*r*b*T)))
    return X0*ncx2.sf(d, n, p) - math.exp(-r*T)*K*ncx2.cdf(p, n-2, d)

In [8]:
def eki_call_control(X0, K, B, T, σ, r, β, N, steps):
    X = milstein(X0, T, σ, r, β, N, steps)
    # ADD CODE HERE
    # simulate the cev vanilla call prices
    cv = math.exp(-r*T)*np.maximum(X-K,0)
    avg_cv = np.average(cv)
    
    KI = X>B
    V = math.exp(-r*T)*np.maximum(X-K,0)*KI
    
    cov = np.cov(cv, V, ddof=1)
    adj = -cov[0,1]/cov[0,0]
    
    price = np.sum(V)/N + adj*(avg_cv - cev_vanilla_call(X0, K, T, σ, r, β))
    stderr = math.sqrt((cov[1,1] - cov[0,1]**2/cov[0,0])/N)
    
    return price, stderr
# print(math.fabs(price2 - 4.77) < 3e-2)

In [9]:
%%time

price2, stderr2 = eki_call_control(X0, K, B, T, σ, r, β, 500000, 100)
assert math.fabs(price2 - 4.77) < 3e-2

Wall time: 22.2 s


(c)	Instead of the barrier being observed at maturity only, we will try and price the same option but with the barrier observed continuously which is called American Knock-In (AKI) in the industry. This means the option will knock in if the asset trades above the barrier at any point during the lifetime of the option. 

Complete two function definitions below:
1. **max_milstein** is a modified version of the milstein function of part a) which will return in additon to the array of final price, the maximum price attained for the simulated path
2. **aki_call** will be used to find the price of the option using the Monte Carlo method.

(40 marks)

In [10]:
def max_milstein(X0, T, σ, r, β, N, steps):
    # ADD CODE HERE
    max_X = X0*np.ones(N)
    Δt = T/steps
    X = X0*np.ones(N)
    prng = np.random.RandomState() 
    z = prng.standard_normal((N, steps)) # create an array of randomly generated numbers. rows for # of paths, col for steps
    for t in range(steps):
        X += r*X*Δt + σ*X**β*math.sqrt(Δt)*z[:,t] + 0.5*β*σ**2*X**(2*β-1)*Δt*(np.power(z[:,t],2)-1)
        max_X = np.maximum(X, max_X)
    
    
    return X, max_X
    

def aki_call(X0, K, B, T, σ, r, β, N, steps):
    X, max_X = max_milstein(X0, T, σ, r, β, N, steps)
    # ADD CODE HERE
    
    KI = max_X>B
    V = math.exp(-r*T)*np.maximum(X-K,0)*KI
    
    # process the price
    price = np.average(V)
    
    # process the stderr
    stderr = np.std(V, ddof=1) / N**0.5
    
    return price, stderr


In [11]:
%%time

X, max_X = max_milstein(X0, T, σ, 0.2, β, 500000, 100)

assert X.shape == (500000,)
assert max_X.shape == (500000,)

Wall time: 22.9 s


In [12]:
%%time

price3, stderr3 = aki_call(X0, K, B, T, σ, r, β, 50000, 2000)
assert math.fabs(price3 - 7.15) < 0.1

Wall time: 44.3 s


In [13]:
print(X.shape)
print(max_X.shape)
print(np.max(max_X))
print(X)
print(max_X)
print(math.fabs(price3 - 7.15))
print(price3)

(500000,)
(500000,)
156.86922921752162
[122.36433089 125.12538225 114.08802345 ... 111.65267997 120.64490825
 122.12932584]
[123.74133325 125.14077104 120.05164625 ... 114.7257344  120.64490825
 123.74579354]
0.03435960993322951
7.18435960993323
